# Використання агентів штучного інтелекту для генерування тестів

### Вступ
Цей проєкт спрямований на створення автоматизованого інструменту для генерації тестових матеріалів на основі текстового вмісту PDF файлів. Інструмент використовує сучасні технології мовних моделей (Large Language Models, LLMs) для аналізу контенту, створення запитань і форматування тестових завдань. Нижче описані основні технології, бібліотеки та концепції, які використовуються в проєкті:

#### Основні технології

1. **Мовні моделі (LLM)**:
   - Великі мовні моделі здатні обробляти текстові дані та генерувати відповідний контент, наприклад, відповіді на питання, резюме тексту або нові запитання на основі аналізу вмісту.
   - У цьому проєкті використовується LLM з API Groq, що дозволяє підключитися до потужних мовних моделей, таких як Llama 3, і застосувати їх для створення тестів.

2. **Бібліотека `langchain_openai`**:
   - `langchain_openai` забезпечує доступ до моделей OpenAI та сумісних платформ. За допомогою цього API ми можемо надсилати запити до моделей, обробляти результати та адаптувати їх під наші цілі.

3. **PyPDF2**:
   - Бібліотека Python для зчитування PDF файлів, дозволяє витягувати текст із кожної сторінки документа. У цьому проєкті `PyPDF2` використовується для конвертації PDF документів у текстовий формат, щоб модель могла проаналізувати текст.

4. **CrewAI**:
   - Бібліотека для створення та управління агентами та завданнями, що дозволяє організовувати процес створення тесту. Використовуючи агенти з різними ролями, такими як "Question Creator" (агент для створення запитань) та "Format Specialist" (агент для форматування), проєкт розподіляє завдання, що підвищує якість та ефективність роботи.

#### Огляд процесу

1. **Отримання тексту з PDF**:
   - Функція `extract_pdf_content` зчитує текст із PDF файлу та об'єднує його в один рядок для подальшого аналізу.

2. **Створення агентів**:
   - `create_agents` створює двох агентів із різними спеціалізаціями: один генерує тестові запитання, а інший відповідає за структурування тесту.

3. **Створення завдань**:
   - `create_tasks` формулює конкретні завдання для кожного агента, забезпечуючи послідовне виконання процесу – спочатку створення запитань, а потім форматування тесту.

4. **Запуск процесу генерації тесту**:
   - Основна функція `generate_test` об'єднує всі етапи: витяг контенту, створення агентів і завдань, та запуск команди для створення кінцевого тесту.

5. **Збереження результату**:
   - Отриманий тест може бути виведений на екран і збережений у файл для подальшого використання.


Проєкт демонструє, як можна автоматизувати створення тестових матеріалів, використовуючи потужні інструменти обробки природної мови. Такий підхід знижує витрати часу на створення навчальних матеріалів, підвищує точність формулювання запитань та забезпечує узгодженість структури тестів.

### 1: Встановлення необхідних пакетів

Для виконання коду в даному нотатнику необхідно переконатися, що всі залежності встановлені. У середовищі Jupyter Notebook команда `%pip install` використовується для встановлення пакетів безпосередньо в середовищі, що дозволяє переконатися, що всі необхідні бібліотеки доступні для коду.

```python
# %pip install -r ../requirements.txt
```

Цей рядок коду встановлює всі необхідні пакети зі списку залежностей у файлі `requirements.txt`. Його можна розкоментувати (прибрати знак `#` на початку рядка), якщо потрібно встановити пакети. Команда закоментована для запобігання повторній установці, якщо пакети вже встановлені.

---

### 2: Налаштування середовища

Наступний крок – налаштування змінних середовища для зберігання конфіденційних даних, таких як API-ключі. Це дозволяє зберігати важливу інформацію окремо від коду, забезпечуючи безпеку.

- `os` – вбудована бібліотека Python для роботи з операційною системою, включаючи управління змінними середовища.
- `dotenv` – бібліотека, що дозволяє завантажувати змінні середовища з `.env` файлу, зберігаючи конфіденційні дані окремо від коду.

```python
import os
from dotenv import load_dotenv

load_dotenv()  # Завантаження змінних середовища з .env файлу
groq_api_key = os.getenv('GROQ_API_KEY')  # Отримання ключа API з середовища
```

- `load_dotenv()` завантажує змінні з `.env` файлу для безпеки при роботі з API. Це дає змогу уникнути прямого введення конфіденційної інформації в код.

### Про проєкт Groq та отримання API-ключа

Groq – це хмарна платформа, яка надає доступ до потужних мовних моделей для обробки тексту, таких як Llama 3, що здатні аналізувати великі обсяги інформації та генерувати текст на основі введеного запиту. Groq API дозволяє інтегрувати ці мовні моделі у власні проєкти, автоматизуючи завдання, пов'язані з обробкою природної мови, наприклад, створення тестів, генерація резюме, класифікація тексту тощо.

#### Отримання API-ключа

Щоб отримати доступ до Groq API:
1. Зареєструйтесь на офіційному вебсайті [Groq](https://groq.com) та створіть обліковий запис.
2. Після входу перейдіть до налаштувань облікового запису або розділу розробників (Developer/API settings).
3. Згенеруйте новий API-ключ, який надасть доступ до мовних моделей.
4. Зберігайте ключ у безпечному місці, а для інтеграції в проєкт використовуйте бібліотеку `dotenv`, щоб зберегти ключ в окремому `.env` файлі.

API-ключ забезпечує безпечний доступ до Groq моделей і є необхідним для авторизації в API при викликах функцій.

### 3: Налаштування мовної моделі (LLM)

Мовні моделі (LLM) дозволяють автоматизувати роботу з текстовими завданнями, такими як генерування тексту, переклад, класифікація тощо. У цьому прикладі ми використовуємо API платформи Groq, щоб отримати доступ до однієї з таких моделей.

#### Опис коду

- **ChatOpenAI**: клас із бібліотеки `langchain_openai`, який дозволяє налаштувати параметри для роботи з моделями OpenAI або сумісними моделями.
- **openai_api_base**: основна URL-адреса для доступу до API Groq.
- **openai_api_key**: API-ключ, необхідний для авторизації (завантажується з середовища для безпеки).
- **model_name**: вибір конкретної моделі, яку ми хочемо використовувати. У цьому прикладі налаштовано `llama3-70b-8192`, модель із 70 мільярдами параметрів, що підтримує до 8192 символів у контексті. Для інших завдань можна спробувати альтернативну модель, наприклад, `mixtral-8x7b-32768`, яка має інші характеристики.

```python
from langchain_openai import ChatOpenAI

# Ініціалізація мовної моделі для роботи через Groq API
llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",  # URL для підключення до API
    openai_api_key=groq_api_key,  # API-ключ для авторизації
    model_name="llama3-70b-8192"  # Вибір моделі: llama3-70b-8192
    # Альтернативна модель: model_name="mixtral-8x7b-32768"
)
```

Цей код налаштовує підключення до мовної моделі через API Groq. Ми використовуємо змінну `llm` для подальшої взаємодії з моделлю, що дозволяє здійснювати запити і отримувати відповіді від моделі.

### 4: Імпорт бібліотек та загальні налаштування

У цьому блоці ми імпортуємо кілька бібліотек, необхідних для різних завдань у програмі. Кожна з них має свою специфічну функцію, яка дозволяє реалізовувати окремі аспекти роботи з даними, штучним інтелектом і обробкою PDF.

#### Опис коду

- **os**: вбудована бібліотека Python, яка надає функції для взаємодії з операційною системою.
- **typing.List**: використовується для анотацій типів, що полегшує розуміння та відлагодження коду.
- **PyPDF2**: бібліотека для роботи з PDF файлами, дозволяє витягувати текст із PDF документів.
- **crewai**: набір класів із `CrewAI`, що дозволяють керувати завданнями та агентами штучного інтелекту.
- **Ollama**: клас із бібліотеки `langchain.llms`, що допомагає керувати мовними моделями, такими як LLM.
- **warnings**: модуль для обробки попереджень, дозволяє фільтрувати зайву інформацію у виводі.

```python
import os
from typing import List
import PyPDF2
from crewai import Agent, Task, Crew
from langchain.llms import Ollama
import warnings

# Відключення попереджень для чистоти виводу
warnings.filterwarnings('ignore')
```

У цьому блоці коду ми імпортуємо всі необхідні бібліотеки. Далі використовується `warnings.filterwarnings('ignore')` для відключення попереджень, що може бути корисним у випадках, коли такі повідомлення не мають суттєвого значення і тільки відволікають користувача.

### 5: Функція для витягання тексту з PDF файлу

Функція `extract_pdf_content` призначена для витягання тексту з PDF файлів. Це корисно для автоматизації завдань, які потребують аналізу тексту, збереженого у PDF документах. Функція зчитує PDF посторінково, об'єднуючи текст у один рядок.

#### Опис коду

- **pdf_path**: параметр функції, що приймає шлях до PDF файлу.
- **content**: змінна, яка збирає текст, витягнутий із кожної сторінки PDF.
- **PyPDF2.PdfReader**: клас для читання PDF файлів, що надає доступ до сторінок і тексту.
- **Exception Handling**: у разі виникнення помилки (наприклад, якщо файл не можна відкрити) генерується повідомлення з деталями помилки.

```python
def extract_pdf_content(pdf_path: str) -> str:
    """Витягує текст із PDF файлу."""
    content = ""  # Ініціалізуємо змінну для зберігання тексту
    try:
        # Відкриваємо PDF файл у режимі читання (rb - read binary)
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)  # Ініціалізація PDF читача
            for page in pdf_reader.pages:  # Перебираємо сторінки PDF
                text = page.extract_text()  # Витягуємо текст з кожної сторінки
                if text:
                    content += text  # Додаємо текст до загальної змінної
        return content  # Повертаємо весь витягнутий текст
    except Exception as e:
        raise Exception(f"Помилка при читанні PDF: {str(e)}")
```

Ця функція відкриває PDF у двійковому режимі читання, використовуючи `PyPDF2` для зчитування вмісту сторінок. Текст із кожної сторінки додається до змінної `content`. У разі виникнення помилки, функція генерує виняток із повідомленням, яке допомагає зрозуміти, що пішло не так.

### 7: Створення агентів для генерації тестів

Цей код створює два спеціалізовані агенти, які допомагають у процесі генерації тестових запитань. Кожен агент має власну роль і ціль, що дозволяє виконувати завдання з більшою точністю та ефективністю. 

#### Опис коду

- **create_agents**: функція, яка приймає мовну модель `llm` як параметр і створює два агенти: `question_creator` і `format_specialist`.
- **Agent**: клас, що представляє агента, якому призначено певну роль і ціль. Параметри агента:
  - **name**: ім'я агента.
  - **role**: конкретна роль, яку виконує агент (наприклад, "test question creator" або "test format specialist").
  - **goal**: мета агента, що визначає його завдання у процесі тестування.
  - **backstory**: бекграунд агента, який описує його навички та компетенції.
  - **llm**: посилання на мовну модель, яку агент використовує для виконання своїх завдань.
  - **verbose**: налаштування, що дозволяє виводити детальну інформацію про роботу агента під час його виконання.

```python
def create_agents(llm):
    """Створення спеціалізованих агентів для генерації тестів."""

    # Агент для створення запитань до тестів
    question_creator = Agent(
        name="Question Creator",
        role="test question creator",
        goal="Create varied and effective test questions from the analyzed content",
        backstory="Expert in creating diverse types of test questions, skilled at assessing different cognitive levels.",
        llm=llm,
        verbose=True
    )

    # Агент для налаштування формату тесту
    format_specialist = Agent(
        name="Format Specialist",
        role="test format specialist",
        goal="Create a well-structured and properly formatted test",
        backstory="Expert in organizing test content and ensuring proper formatting.",
        llm=llm,
        verbose=True
    )

    # Повертаємо обидва агенти для подальшого використання
    return question_creator, format_specialist
```

#### Пояснення коду

1. **question_creator** – агент, який генерує різноманітні та ефективні тестові питання. Його роль і ціль орієнтовані на створення запитань, що оцінюють різні рівні когнітивних навичок.
2. **format_specialist** – агент, який відповідає за структурування та форматування тесту. Його мета – забезпечити чітке та логічне розташування тестових елементів.
3. **Повернення значень** – обидва агенти повертаються для подальшого використання в програмі, наприклад, для створення повноцінного тесту.

Цей підхід дозволяє розподілити завдання між спеціалізованими агентами, що підвищує ефективність і якість створення тестових матеріалів.

### 8: Створення завдань для генерації тесту

Функція `create_tasks` створює послідовні завдання для агентів, які генерують питання до тесту та структурують його. Завдання передаються відповідним агентам для забезпечення ефективного виконання.

#### Опис коду

- **create_tasks**: функція, яка приймає текстовий вміст PDF, агента для створення запитань (`question_creator`) та агента для форматування (`format_specialist`).
- **Task**: клас, який представляє окреме завдання для агента. Параметри завдання:
  - **description**: опис завдання, що включає інструкції для агента.
  - **agent**: агент, відповідальний за виконання завдання.
  - **dependencies**: список завдань, які мають бути виконані раніше (для завдань з послідовною залежністю).
  - **expected_output**: очікуваний результат завдання у визначеному форматі.

```python
def create_tasks(pdf_content: str, question_creator, format_specialist) -> List[Task]:
    """Створення послідовних завдань для генерації тесту."""

    # Завдання для створення питань на основі аналізу PDF-контенту
    generate_questions_task = Task(
        description=f"""
        Analyze the following content and create 20 test questions:
        {pdf_content[:1000]}...  # Скорочений текст для зручності
        Ensure questions:
        - Cover main topics and key concepts
        - Include different difficulty levels
        - Test different cognitive skills
        """,
        agent=question_creator,
        expected_output="List of test questions in JSON format."
    )

    # Завдання для форматування тесту
    format_test_task = Task(
        description="Organize questions by type, add instructions, and ensure consistent JSON formatting.",
        agent=format_specialist,
        dependencies=[generate_questions_task],
        expected_output="A well-formatted test ready for review."
    )

    # Повернення списку завдань
    return [generate_questions_task, format_test_task]
```

#### Пояснення коду

1. **generate_questions_task** – завдання для створення тестових питань на основі аналізу тексту, взятого з PDF. Опис включає інструкції щодо різних рівнів складності та тестування когнітивних навичок. Частина тексту `pdf_content[:1000]` скорочена для зручності.
2. **format_test_task** – завдання для форматування тесту, яке залежить від виконання попереднього завдання `generate_questions_task`. Це завдання відповідає за структурування питань, додавання інструкцій та перевірку формату JSON.
3. **dependencies** – залежність, що забезпечує послідовне виконання завдань. `format_test_task` виконується лише після завершення `generate_questions_task`.
4. **Повернення завдань** – обидва завдання повертаються у вигляді списку для подальшого використання.

Цей підхід дозволяє чітко структурувати процес створення тестів, забезпечуючи послідовне виконання завдань і підвищуючи якість кінцевого результату.

### 9: Основна функція генерації тесту

Функція `generate_test` є основною для виконання всього процесу генерації тесту. Вона об’єднує всі попередні функції та методи, включаючи витяг тексту з PDF, створення агентів і завдань, а також запуск команди агентів для виконання завдань.

#### Опис коду

- **generate_test**: функція, яка приймає шлях до PDF файлу та виконує повний процес генерації тесту.
  - **extract_pdf_content**: викликає функцію для витягання тексту з PDF файлу.
  - **create_agents**: створює агентів для генерації тестових запитань і форматування тесту.
  - **create_tasks**: створює завдання для агентів на основі вмісту PDF.
  - **Crew**: об'єднує агентів і завдання в єдину команду для виконання процесу тестування.
  - **kickoff**: запускає команду агентів на виконання завдань.
  - **result**: змінна, яка зберігає кінцевий результат у вигляді згенерованого тесту або повідомлення про помилку.

```python
def generate_test(pdf_path: str) -> str:
    """Виконання процесу генерації тесту."""
    try:
        # Витяг вмісту PDF файлу
        pdf_content = extract_pdf_content(pdf_path)
        
        # Створення агентів
        question_creator, format_specialist = create_agents(llm)
        
        # Створення завдань
        tasks = create_tasks(pdf_content, question_creator, format_specialist)
        
        # Створення та запуск команди агентів
        crew = Crew(
            agents=[question_creator, format_specialist],
            tasks=tasks,
            verbose=True
        )
        
        # Запуск процесу
        result = crew.kickoff()
        
        return result
    
    except Exception as e:
        raise Exception(f"Error generating test: {str(e)}")
```

#### Пояснення коду

1. **pdf_content** – змінна, яка отримує текстовий вміст PDF за допомогою функції `extract_pdf_content`.
2. **create_agents** – створює агентів `question_creator` і `format_specialist`, що відповідають за створення питань та форматування тесту.
3. **create_tasks** – створює завдання для агентів на основі тексту PDF.
4. **Crew** – об’єднує створених агентів і завдання у команду, яка працює над створенням тесту. Параметр `verbose=True` забезпечує виведення детальної інформації про виконання.
5. **kickoff** – запускає команду агентів для виконання завдань.
6. **result** – результат у вигляді завершеного тесту або повідомлення про помилку у випадку невдачі.

Функція повертає результат у вигляді сформованого тесту. У разі виникнення помилки, вона генерує виняток з повідомленням про причину помилки, що допомагає виявити проблеми в процесі генерації тесту.

### 10: Приклад використання функції генерації тесту

Ця частина коду показує, як викликати основну функцію `generate_test` для створення тесту на основі PDF файлу. В результаті отриманий тест може бути виведений на екран або збережений у текстовий файл.

#### Опис коду

- **PDF_PATH**: змінна, яка містить шлях до PDF файлу, що використовується для генерації тесту.
- **generate_test(PDF_PATH)**: виклик основної функції генерації тесту.
- **try-except**: блок для обробки помилок, який дозволяє вивести повідомлення про помилку у випадку невдачі.
- **open("generated_test.txt", "w")**: зберігає згенерований тест у текстовий файл.

```python
# Вказуємо шлях до PDF файлу
PDF_PATH = "data/Lowther.pdf"

# Генерація тесту
try:
    test = generate_test(PDF_PATH)
    print("Test generated successfully!")
    print("\nGenerated Test:")
    print("-" * 50)
    print(test)
    
    # Опціонально: збереження тесту у файл
    with open("generated_test.txt", "w") as f:
        f.write(test)
    print("\nTest saved to 'generated_test.txt'")
    
except Exception as e:
    print(f"Error: {str(e)}")
```

#### Пояснення коду

1. **PDF_PATH** – змінна містить шлях до PDF файлу, який буде оброблено функцією `generate_test`.
2. **generate_test(PDF_PATH)** – виклик функції, яка виконує весь процес генерації тесту на основі заданого PDF файлу.
3. **print** – виводить повідомлення про успішне створення тесту та сам текст тесту.
4. **Збереження у файл** – тест зберігається у файл `"generated_test.txt"` для подальшого використання. Файл відкривається в режимі запису (`"w"`), і текст тесту записується у файл.
5. **Обробка помилок** – у разі виникнення помилки, наприклад, якщо файл не знайдено або виникла помилка у функції, виводиться повідомлення з деталями помилки.

Цей приклад показує, як автоматизувати процес створення тесту, починаючи з аналізу PDF файлу і закінчуючи збереженням результату.

### Висновки

Цей проєкт продемонстрував, як за допомогою сучасних технологій обробки природної мови можна автоматизувати створення тестових матеріалів на основі текстового вмісту PDF файлів. Використовуючи мовні моделі, бібліотеки для роботи з PDF, та систему агентів для генерації та форматування тестів, ми досягли низки переваг:

1. **Автоматизація процесу**: Інструмент суттєво знижує витрати часу на створення тестових питань і форматування тестів, що може бути особливо корисним для освітніх установ та організацій.

2. **Підвищення якості тестів**: Завдяки можливостям мовної моделі тестові питання створюються з урахуванням когнітивних рівнів складності, а також охоплюють ключові теми та концепції, що підвищує ефективність тестування.

3. **Структурована організація**: Використання агентів із конкретними ролями дозволяє розподілити процес на окремі завдання, що забезпечує узгодженість і якість результату. Форматування тестів відповідно до визначених стандартів робить їх зручними для використання.

4. **Гнучкість та масштабованість**: Проєкт легко адаптувати під різні джерела інформації, тематики та види тестів, що робить його універсальним рішенням для різних потреб.

Таким чином, цей інструмент є прикладом того, як можна застосовувати потужні алгоритми штучного інтелекту для автоматизації процесів у сфері освіти та підвищення ефективності роботи.